# Exercise: Analyzing Chinook Database

Preparation I've done:
 - Retrieve the dataset and load it
 - Load the %sql extension and point it at the database
 - Display the tables and an example query

Additional steps you might take:
 - Add libraries for visualization (matplotlib, seaborn, plotly)
 - Add libraries for statistics (numpy)
 - Explore the dataset using SQL and/or pandas

----

1. Retrieve a list of all the tracks in the database, displaying only the track name and the name of the album it belongs to. Limit the result to the first 5 rows.
   > Operations: `SELECT`
2. Find the total number of customers from each country. Display the country name and the corresponding count. Order the results by the count in descending order.
   > Operations: `SELECT`, `COUNT`, `GROUP BY`, `ORDER BY`
3. Identify the top 5 genres with the highest number of tracks. Display the genre name along with the total number of tracks for each genre.
   > Operations: `SELECT`, `COUNT`, `GROUP BY`, `ORDER BY`
4. Determine the average invoice total for each customer, considering both the album and individual track purchases. Display the customer's first and last name along with the average invoice total. Order the results by the average invoice total in descending order.
   > Operations: `SELECT`, `AVG`, `JOIN`, `GROUP BY`, `ORDER BY`
5. Identify the customer who spent the most on music purchases. Display the customer's first and last name, along with the total amount spent.
   > Operations: `SELECT`, `SUM`, `JOIN`, `GROUP BY`, `ORDER BY`, `LIMIT`

In [16]:
#%pip install pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.2/27.2 MB 12.9 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
# Load chinook dataset and query it using SQL magic into pandas dataframes
import pandas as pd
import pyarrow as pa
import sqlite3
%load_ext sql

# Load data
conn = sqlite3.connect("chinook.sqlite")

# Tell %sql about the database
%sql sqlite:///chinook.sqlite

# List tables in database
query = "SELECT name FROM sqlite_master WHERE type='table';"

# Read data into a Pandas DataFrame
tables = %sql $query

# Print head
display(tables)

# Query to get the first 5 rows of the `albums` table
result = %sql SELECT * FROM tracks LIMIT 5;

# Display query result, note that Pandas DataFrame is returned!
display(result)


The sql extension is already loaded. To reload it, use:
  %reload_ext sql
 * sqlite:///chinook.sqlite
Done.


name
albums
sqlite_sequence
artists
customers
employees
genres
invoices
invoice_items
media_types
playlists


 * sqlite:///chinook.sqlite
Done.


TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman",230619,3990994,0.99
4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. Dirkscneider & W. Hoffman",252051,4331779,0.99
5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99


1. Retrieve a list of all the tracks in the database, displaying only the track name and the name of the album it belongs to. Limit the result to the first 5 rows.
   > Operations: `SELECT`

In [6]:
result = %sql SELECT t.Name as TrackName, a.Title as AlbumName \
        from  tracks t JOIN albums a USING(albumid) LIMIT 5;

display(result)

 * sqlite:///chinook.sqlite
Done.


TrackName,AlbumName
For Those About To Rock (We Salute You),For Those About To Rock We Salute You
Put The Finger On You,For Those About To Rock We Salute You
Let's Get It Up,For Those About To Rock We Salute You
Inject The Venom,For Those About To Rock We Salute You
Snowballed,For Those About To Rock We Salute You


2. Find the total number of customers from each country. Display the country name and the corresponding count. Order the results by the count in descending order.
   > Operations: `SELECT`, `COUNT`, `GROUP BY`, `ORDER BY`

In [18]:
result = %sql SELECT Country, COUNT(CustomerId) as CustomerCount \
FROM customers \
GROUP BY Country \
ORDER BY CustomerCount DESC;

# Display query result, note that Pandas DataFrame is returned!
display(result)

 * sqlite:///chinook.sqlite
Done.


Country,CustomerCount
USA,13
Canada,8
France,5
Brazil,5
Germany,4
United Kingdom,3
Portugal,2
India,2
Czech Republic,2
Sweden,1


3. Identify the top 5 genres with the highest number of tracks. Display the genre name along with the total number of tracks for each genre.
   > Operations: `SELECT`, `COUNT`, `GROUP BY`, `ORDER BY`


In [8]:
result = %sql SELECT g.Name as GenreName, COUNT(t.TrackId) as TrackCount \
FROM genres g \
JOIN tracks t USING(GenreId) \
GROUP BY GenreName \
ORDER BY TrackCount DESC \
LIMIT 5;

# Display query result, note that Pandas DataFrame is returned!
display(result)

 * sqlite:///chinook.sqlite
Done.


GenreName,TrackCount
Rock,1297
Latin,579
Metal,374
Alternative & Punk,332
Jazz,130


4. Determine the average invoice total for each customer, considering both the album and individual track purchases. Display the customer's first and last name along with the average invoice total. Order the results by the average invoice total in descending order.
   > Operations: `SELECT`, `AVG`, `JOIN`, `GROUP BY`, `ORDER BY`


In [14]:
result = %sql SELECT c.FirstName, c.LastName, ROUND(AVG(i.Total),2) as AverageInvoiceTotal \
FROM customers c \
JOIN invoices i ON c.CustomerId = i.CustomerId \
GROUP BY c.CustomerId \
ORDER BY AverageInvoiceTotal DESC;

display(result)

 * sqlite:///chinook.sqlite
Done.


FirstName,LastName,AverageInvoiceTotal
Helena,Holý,7.09
Richard,Cunningham,6.8
Luis,Rojas,6.66
Ladislav,Kovács,6.52
Hugh,O'Reilly,6.52
Frank,Ralston,6.23
Julia,Barnett,6.23
Fynn,Zimmermann,6.23
Puja,Srivastava,6.11
Astrid,Gruber,6.09


5. Identify the customer who spent the most on music purchases. Display the customer's first and last name, along with the total amount spent.
   > Operations: `SELECT`, `SUM`, `JOIN`, `GROUP BY`, `ORDER BY`, `LIMIT`

In [13]:
result = %sql SELECT c.FirstName, c.LastName, ROUND(SUM(i.Total),2) as TotalAmountSpent \
FROM customers c \
JOIN invoices i ON c.CustomerId = i.CustomerId \
GROUP BY c.CustomerId \
ORDER BY TotalAmountSpent DESC \
LIMIT 1;

display(result)


 * sqlite:///chinook.sqlite
Done.


FirstName,LastName,TotalAmountSpent
Helena,Holý,49.62


In [17]:
%%sql

-- Query 1: Select top 5 genres with the highest number of tracks
SELECT g.Name as GenreName, COUNT(t.TrackId) as TrackCount
FROM genres g
JOIN tracks t USING(GenreId)
GROUP BY GenreName
ORDER BY TrackCount DESC
LIMIT 5;

-- Query 2: Identify the customer who spent the most on music purchases
SELECT c.FirstName, c.LastName, SUM(i.Total) as TotalAmountSpent
FROM customers c
JOIN invoices i ON c.CustomerId = i.CustomerId
GROUP BY c.CustomerId
ORDER BY TotalAmountSpent DESC
LIMIT 1;


 * sqlite:///chinook.sqlite
Done.
Done.


FirstName,LastName,TotalAmountSpent
Helena,Holý,49.620000000000005


In [20]:
result = %sql SELECT * FROM invoices
display(result)

 * sqlite:///chinook.sqlite
Done.


InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
3,8,2009-01-03 00:00:00,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86
6,37,2009-01-19 00:00:00,Berger Straße 10,Frankfurt,None,Germany,60316,0.99
7,38,2009-02-01 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,1.98
8,40,2009-02-01 00:00:00,"8, Rue Hanovre",Paris,None,France,75002,1.98
9,42,2009-02-02 00:00:00,"9, Place Louis Barthou",Bordeaux,None,France,33000,3.96
10,46,2009-02-03 00:00:00,3 Chatham Street,Dublin,Dublin,Ireland,None,5.94
